<a href="https://colab.research.google.com/github/Bhagyashree2k2/ADA-pograms/blob/main/Bhagyashree_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
directory_path='/content/drive/MyDrive/Brain Tumor/'
os.chdir(directory_path)
os.getcwd()

'/content/drive/MyDrive/Brain Tumor'

In [ ]:
path = '/content/drive/MyDrive/Brain Tumor/Brain_MRI/'


In [ ]:
pip install pandas

In [ ]:
pip install numpy


In [ ]:
pip install matplotlib

In [ ]:
pip install seaborn

In [ ]:
pip install opencv-python

In [ ]:
pip install scikit-learn

In [ ]:
pip install tensorflow

In [ ]:
pip install keras

In [ ]:
pip install Keras-Preprocessing

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import skimage.io as io
import cv2

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers,optimizers
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
import random

In [ ]:
%matplotlib inline
#to display plots in the same file

In [ ]:
brain_df=pd.read_csv('/content/drive/MyDrive/Brain Tumor/Brain_MRI/data_mask.csv')


In [ ]:
brain_df

In [ ]:
brain_df.info()


In [ ]:
brain_df.image_path[0]

In [ ]:
brain_df.mask_path[0]

In [ ]:
brain_df['mask'].value_counts()

In [ ]:
fig=plt.figure(figsize=(6,5))
plt.bar(x=['0','1'],height=brain_df['mask'].value_counts())
plt.show()

In [ ]:
image=cv2.imread(path+brain_df.image_path[1000])
plt.imshow(image)

In [ ]:
image=cv2.imread(path+brain_df.mask_path[1000])
plt.imshow(image)

In [ ]:
cv2.imread(path+brain_df.image_path[1000]).max()

In [ ]:
cv2.imread(path+brain_df.image_path[1000]).min()

In [ ]:
cv2.imread(path+brain_df.mask_path[1000]).max()

In [ ]:
cv2.imread(path+brain_df.mask_path[1000]).min()

In [ ]:
fig,axs=plt.subplots(3,2,figsize=(8,12))
count=0
for x in range(3):
  i=random.randint(0,len(brain_df))
  axs[count][0].title.set_text("Brain MRI")
  axs[count][0].imshow(cv2.imread(path + brain_df.image_path[i]))
  axs[count][1].title.set_text("Mask")
  axs[count][1].imshow(cv2.imread(path + brain_df.mask_path[i]))
  count=count+1
fig.tight_layout()

preprocessing

In [ ]:
brain_df.head(5)

In [ ]:
brain_df.shape

In [ ]:
b_df=brain_df.drop(columns=['patient_id'])
b_df.shape

In [ ]:
b_df.info()

In [ ]:
#converting mask column into object datatype
b_df['mask']=b_df['mask'].apply(lambda x:str(x))

In [ ]:
b_df.info()

In [ ]:
from sklearn.model_selection import train_test_split
train,test=train_test_split(b_df,test_size=0.15)

In [ ]:
test

In [ ]:
train

In [ ]:
#for data augmentation-cropping,padding
from keras_preprocessing.image import ImageDataGenerator

datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.15)
#validation split will help in measuring accuracy for a particular batch

In [ ]:
train_generator = datagen.flow_from_dataframe(
    dataframe=train,
    directory=path,
    x_col='image_path',
    y_col='mask',
    subset="training",
    batch_size=16,
    shuffle=False,
    class_mode="categorical",
    target_size=(256,256)
)
valid_generator = datagen.flow_from_dataframe(
    dataframe=train,
    directory=path,
    x_col='image_path',
    y_col='mask',
    subset="validation",
    batch_size=16,
    shuffle=False,
    class_mode="categorical",
    target_size=(256,256)
)

In [ ]:
test_datagen=ImageDataGenerator(rescale=1./255.)
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test,
    directory=path,
    x_col='image_path',
    y_col='mask',
    subset="training",
    batch_size=32,
    shuffle=False,
    class_mode="categorical",
    target_size=(256,256)
)

In [ ]:
 #Transfer learning using a pretrained CNN model (ResNet50)
 basemodel=ResNet50(weights='imagenet',include_top=False,input_tensor=Input(shape=(256,256,3)))

In [ ]:
basemodel.summary()

In [ ]:
plot_model(basemodel,to_file='/content/drive/MyDrive/Brain Tumor/model_plot.png',show_shapes=True,show_layer_names=True)

In [ ]:
#freezing the layer
for layers in basemodel.layers:
  layers.trainable=False

In [ ]:
#adding layers to pretrained network
headmodel=basemodel.output
headmodel=MaxPooling2D(pool_size=(4,4))(headmodel)
headmodel=Flatten(name='flatten')(headmodel) #converting output from pooling layer to 1D (flattening)
headmodel=Dense(256,activation='relu')(headmodel)#Dense is a fully connected layer used to identify the classes i.e., classification
headmodel=Dropout(0.3)(headmodel)
headmodel=Dense(256,activation='relu')(headmodel)
#dense(unit,activation-relu(fxn)-rectified linear unit-to learn complex parameters in the data)
headmodel=Dropout(0.3)(headmodel)
headmodel=Dense(256,activation='relu')(headmodel)
headmodel=Dropout(0.3)(headmodel)
headmodel=Dense(2,activation='softmax')(headmodel)#softmax is activation fxn that provides the probability of each classs

model=Model(inputs=basemodel.input,outputs=headmodel)


In [ ]:
#bias  #variance  
#low variance & high bias -- underfitting
#high variance and low bias -- Overfitting
#To prevent overfitting we add dropout layers
#To temperarily remove neurons from training to avoid overfitting (here 30%)



In [ ]:
model.summary()

In [ ]:
#compiling the model before training the model to define loss function,optimizers and metrics for prediction
#optimizers are algorithms/fxns to change attributes of neural network like weights and learning rate to reduce loss
#optimizers are used to reduce loss
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
#callback is fxn to be applied at certain stages of training
earlystopping=EarlyStopping(monitor='val_loss',verbose=1,patience=25)

checkpointer=ModelCheckpoint(filepath='classifier-resnet-weights.hdf5',mode='min',verbose=1,save_best_only=True)

In [ ]:
tf.test.gpu_device_name()

In [ ]:
#model training
history=model.fit(train_generator,steps_per_epoch=train_generator.n//32,epochs=30,validation_data=valid_generator,callbacks=[earlystopping,checkpointer])

In [ ]:
model_json=model.to_json()
with open("resnet.json","w") as json_file:
  json_file.write(model_json)

In [ ]:
with open('resnet.json','r') as json_file:
  savedmodel=json_file.read()
model=tf.keras.models.model_from_json(savedmodel)

In [ ]:
model.load_weights('classifier-resnet-weights.hdf5')
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
test_predict=model.predict(test_generator,steps=test_generator.n//16,verbose=1)

In [ ]:
test_predict.shape

In [ ]:
test_predict

In [ ]:
y_classes=[]
for i in test_predict:
  y_classes.append(str(np.argmax(i)))

y_classes=np.array(y_classes)

In [ ]:
original=np.array(test['mask'])[:len(test_predict)]

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy=accuracy_score(original,y_classes)

In [ ]:
accuracy

In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(original,y_classes)
plt.figure(figsize=(6,6))
sns.heatmap(cm,annot=True)